In [1]:
%pip install python-dotenv

In [4]:
!pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.0 MB/s eta 0:00:00


In [8]:
# Setup
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

# Using Low Temp for consistent comparison
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.0)

Enter your Google API Key: ··········


In [9]:
# The Task: Reject a candidate for a job.
task = "Write a rejection email to a candidate."

print("--- LAZY PROMPT ---")
print(llm.invoke(task).content)

--- LAZY PROMPT ---
Here are a few options for a rejection email, ranging from a standard template to one for a candidate who interviewed. Choose the one that best fits your situation.

---

**Option 1: Standard Rejection (No Interview)**

This is suitable for candidates who applied but were not selected for an interview.

**Subject: Update on Your Application for [Job Title] at [Company Name]**

Dear [Candidate Name],

Thank you for your interest in the [Job Title] position at [Company Name] and for taking the time to submit your application.

We received a large number of highly qualified applications for this role. While your qualifications are impressive, we have decided to move forward with other candidates whose profiles were a closer match for the specific requirements of this position at this time.

We appreciate you considering [Company Name] as a potential employer and wish you the best of luck in your job search and future endeavors.

Sincerely,

[Your Name]
[Your Title]
[Co

In [10]:
structured_prompt = """
# Context
You are an HR Manager at a quirky startup called 'RocketBoots'.

# Objective
Write a rejection email to a candidate named Bob.

# Constraints
1. Be extremely brief (under 50 words).
2. Do NOT say 'we found someone better'. Say 'the role changed'.
3. Sign off with 'Keep flying'.

# Output Format
Plain text, no subject line.
"""

print("--- STRUCTURED PROMPT ---")
print(llm.invoke(structured_prompt).content)

--- STRUCTURED PROMPT ---
Hi Bob,

Thank you for your interest in RocketBoots.

We appreciate you taking the time to interview. However, the role's requirements have changed significantly since your application. We wish you the best in your job search.

Keep flying.


In [16]:
# ===== Assignment: Part 2a – Structured Prompt for Code Generation =====

structured_prompt = """
# Context
You are a Senior Python Developer with strong software engineering standards.

# Objective
Write a Python function that reverses a string.

# Constraints
1. You MUST use recursion.
2. Do NOT use slicing (no [::-1]).
3. The function must handle empty strings correctly.

# Style
1. Include a detailed Python docstring.
2. Follow clean, readable Python conventions.

# Output Format
Provide only valid Python code.
"""

print(llm.invoke(structured_prompt).content)


```python
def reverse_string(s: str) -> str:
    """
    Reverses a string using a recursive helper function, strictly without using any string slicing.

    This function provides a recursive implementation to reverse a string.
    It rigorously adheres to the constraint of not using any form of string slicing
    (e.g., `s[1:]`, `s[:-1]`, or `s[::-1]`). Instead, it employs a nested helper
    function that builds the reversed string character by character by iterating
    from the end of the original string towards the beginning.

    The main `reverse_string` function handles the base cases for empty or
    single-character strings directly and then delegates the recursive work
    to an internal helper function.

    Args:
        s: The input string to be reversed.

    Returns:
        The reversed string.

    Constraints:
    1. Must use recursion.
    2. Must NOT use string slicing (e.g., `s[1:]`, `s[:-1]`, `s[::-1]`).
    3. Must handle empty strings correctly.

    Examples:

In [11]:
# Setup
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.5)

In [12]:
prompt_zero = "Combine 'Angry' and 'Hungry' into a funny new word."
print(f"Zero-Shot: {llm.invoke(prompt_zero).content}")

Zero-Shot: The perfect word for that is **Hangry**!

It's already a widely recognized and funny term for being irritable due to hunger.


In [13]:
prompt_few = """
Combine words into a funny new word. Give a sarcastic definition.

Input: Breakfast + Lunch
Output: Brunch (An excuse to drink alcohol before noon)

Input: Chill + Relax
Output: Chillax (What annoying people say when you are panic attacks)

Input: Angry + Hungry
Output:
"""
print(f"Few-Shot: {llm.invoke(prompt_few).content}")

Few-Shot: Input: Angry + Hungry
Output: Hangry (A completely valid medical condition that justifies all your irrational behavior until someone feeds you.)


In [14]:
# Setup
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [15]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# 1. Our Database of Examples
examples = [
    {"input": "The internet is down.", "output": "We are observing connectivity latency."},
    {"input": "This code implies a bug.", "output": "The logic suggests unintended behavior."},
    {"input": "I hate this feature.", "output": "This feature does not align with my preferences."},
]

# 2. Template for ONE example
example_fmt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}")
])

# 3. The Few-Shot Container
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_fmt,
    examples=examples
)

# 4. The Final Chain
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Corpo-Speak Translator. Rewrite the input to sound professional."),
    few_shot_prompt,      # Inject examples here
    ("human", "{text}")
])

chain = final_prompt | llm

print(chain.invoke({"text": "This app sucks."}).content)

This application presents several opportunities for enhancement.
